In [130]:
pip install -r ../requirements.txt --quiet

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [131]:
import json
from tqdm import tqdm
from razdel import sentenize
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

In [133]:
device = 'mps' if torch.backends.mps.is_available() else 'cpu'
model_name = "sarahai/ru-sum"

In [134]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
model.eval()

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 512)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 512)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
          

In [192]:
# def abstractive_summary(text, max_symbols=300):
#     inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(device)
#     summary_ids = model.generate(
#           **inputs,
#           max_length=100,
#           min_length=60,
#           num_beams=30,
#           repetition_penalty=3.0,
#           length_penalty=1.2,
#           no_repeat_ngram_size=10,
#           early_stopping=True
#       )
#     summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
#     return summary[:max_symbols]


def improved_abstractive_summary(text, max_symbols=300, device='cuda'):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to('mps')
    with torch.no_grad():
        summary_ids = model.generate(
            **inputs,
            max_length=100,          # подогнал под размер 300 на глаз
            min_length=60,
            num_beams=30,            # взял прямо жирно, но так мне больше нравится
            temperature=0.9,         # Для чуть большей вариативности
            repetition_penalty=1.5,  # Мягче штрафуем повторения
            length_penalty=1.0,      # Нейтральное влияние на длину
            no_repeat_ngram_size=3,  
            early_stopping=True,
            do_sample=True          
        )
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary[:max_symbols]


In [221]:
with open("ex2.json", "r", encoding="utf-8") as f:
    texts = json.load(f)

summaries = []
for text in tqdm(texts, desc="Summarizing texts"):
    abs_summary = improved_abstractive_summary(text)
    summaries.append(abs_summary)

with open("summaries.json", "w", encoding="utf-8") as f:
    json.dump(summaries, f, ensure_ascii=False, indent=2)

Summarizing texts: 100%|██████████| 5/5 [01:20<00:00, 16.02s/it]


In [227]:
reference_summaries = [
    "В избушке живёт пёс Дик, который любит наблюдать, как курят. Он обжорлив и любит есть рыбью требуху. Однажды автор собрал чернику на болоте и пытался научить Дика уму, показывая, как есть ягоды с куста. Через два дня пёс собрал чернику вокруг избушки, чему автор был рад.",
    "Скворец Петруша любит подражать голосам и звукам. Поэт сочинял стихи и печатал их в журналах. В песне Петруши есть трели, свисты и рулады, которые звучат как весенние природные явления, принося радость и настроение весны.",
    "Зайцы обычно не собирают букеты, ведь у каждого из них есть свой цветок — хвост. Но один заяц собрал настоящий букет и не знает, кому его подарить. Лисе и волку цветы не нужны, мишка любит ягоды, а барсук может даже наброситься. Решают отдать букет барсуку и посмотреть, что будет.",
    "Ночью у костра главного героя охватывает страх. Ему кажется, что кто-то наблюдает из темноты и шепчется вокруг. После громкого крика появляется шум листьев — это шевеление листобоя в кронах деревьев. Рассказ передает атмосферу тревоги и таинственности леса.",
    "В ночь в печной трубе задул листобой — холодный октябрьский ветер. Форточка была уже раскрыта настежь и полна берёзовых листьев. Листья сбрасывала берёза под окном, а сам хозяин уже скрылся куда-то."
]

for i in range(len(texts)):
    print(f"Длинна исходного текста: {len(texts[i])}, Длинна реферата: {len(summaries[i])}, Длинна референса: {len(reference_summaries[i])}")

Длинна исходного текста: 1054, Длинна реферата: 167, Длинна референса: 271
Длинна исходного текста: 1101, Длинна реферата: 186, Длинна референса: 221
Длинна исходного текста: 807, Длинна реферата: 196, Длинна референса: 281
Длинна исходного текста: 684, Длинна реферата: 170, Длинна референса: 257
Длинна исходного текста: 740, Длинна реферата: 246, Длинна референса: 198


In [228]:
from rouge import Rouge

rouge = Rouge()

for i, (summary, reference) in enumerate(zip(summaries, reference_summaries)):
    score = rouge.get_scores(summary, reference, avg=False)
    print(f"\nРезультаты для пары {i+1}:")
    print(f"ROUGE-1: {score[0]['rouge-1']}")
    print(f"ROUGE-2: {score[0]['rouge-2']}")
    print(f"ROUGE-L: {score[0]['rouge-l']}")



Результаты для пары 1:
ROUGE-1: {'r': 0.21052631578947367, 'p': 0.2857142857142857, 'f': 0.24242423753902673}
ROUGE-2: {'r': 0.045454545454545456, 'p': 0.06896551724137931, 'f': 0.05479451575905465}
ROUGE-L: {'r': 0.21052631578947367, 'p': 0.2857142857142857, 'f': 0.24242423753902673}

Результаты для пары 2:
ROUGE-1: {'r': 0.5161290322580645, 'p': 0.5925925925925926, 'f': 0.5517241329548158}
ROUGE-2: {'r': 0.3939393939393939, 'p': 0.48148148148148145, 'f': 0.43333332838333327}
ROUGE-L: {'r': 0.5161290322580645, 'p': 0.5925925925925926, 'f': 0.5517241329548158}

Результаты для пары 3:
ROUGE-1: {'r': 0.1590909090909091, 'p': 0.23333333333333334, 'f': 0.18918918436815207}
ROUGE-2: {'r': 0.020833333333333332, 'p': 0.034482758620689655, 'f': 0.02597402127846265}
ROUGE-L: {'r': 0.1590909090909091, 'p': 0.23333333333333334, 'f': 0.18918918436815207}

Результаты для пары 4:
ROUGE-1: {'r': 0.25, 'p': 0.3, 'f': 0.27272726776859507}
ROUGE-2: {'r': 0.05555555555555555, 'p': 0.06896551724137931, '